# Filtro per Fake News
Il problema delle fake news è cresciuto esponenzialmente nell'ultimo decennio a causa della crescente diffusione dei social network, il governo degli Stati Uniti ha deciso di muoversi a tal proposito, incaricando la tua azienda di realizzare un plug-in per chrome in grado di riconoscere se una notizia è falsa. Il tuo compito è quello di realizzare il modello in grado di riconoscere le notizie false, che poi il team di machine learning enginner e web developer metterà in produzione. Ti vengono messi a disposizioni due raccolte di notizie, una contenente solo notizie false e l'altra contenente solo notizie vere, utilizzale per addestrare il tuo modello.

### [Link al dataset](https://proai-datasets.s3.eu-west-3.amazonaws.com/fake_news.zip)

Parti da un'accurata analisi, rispondendo a domande come:
- le fake news sono più frequenti in una determinata categoria?
- per ogni categoria, ci sono argomenti che sono più soggetti alle fake news?
- I titoli delle fake news presentano dei pattern?

Una volta addestrato il modello esportalo [utilizzando pickle](https://scikit-learn.org/stable/model_persistence.html) così che i tuoi colleghi possano metterlo in produzione.

## Importiamo i dataset

In [ ]:
!wget https://proai-datasets.s3.eu-west-3.amazonaws.com/fake_news.zip
!unzip fake_news.zip

--2023-10-03 17:52:41--  https://proai-datasets.s3.eu-west-3.amazonaws.com/fake_news.zip
Resolving proai-datasets.s3.eu-west-3.amazonaws.com (proai-datasets.s3.eu-west-3.amazonaws.com)... 16.12.18.34, 52.95.154.32
Connecting to proai-datasets.s3.eu-west-3.amazonaws.com (proai-datasets.s3.eu-west-3.amazonaws.com)|16.12.18.34|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 42975911 (41M) [application/zip]
Saving to: ‘fake_news.zip’

fake_news.zip       100%[===================>]  40.98M  23.4MB/s    in 1.8s    

2023-10-03 17:52:44 (23.4 MB/s) - ‘fake_news.zip’ saved [42975911/42975911]

Archive:  fake_news.zip
  inflating: Fake.csv                
  inflating: True.csv                


In [ ]:
import pandas as pd

In [ ]:
df_true = pd.read_csv("True.csv")
df_true

,title,text,subject,date
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017"
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017"
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017"
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017"
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017"
...,...,...,...,...
21412,'Fully committed' NATO backs new U.S. approach...,BRUSSELS (Reuters) - NATO allies on Tuesday we...,worldnews,"August 22, 2017"
21413,LexisNexis withdrew two products from Chinese ...,"LONDON (Reuters) - LexisNexis, a provider of l...",worldnews,"August 22, 2017"
21414,Minsk cultural hub becomes haven from authorities,MINSK (Reuters) - In the shadow of disused Sov...,worldnews,"August 22, 2017"
21415,Vatican upbeat on possibility of Pope Francis ...,MOSCOW (Reuters) - Vatican Secretary of State ...,worldnews,"August 22, 2017"


In [ ]:
df_fake = pd.read_csv("Fake.csv")
df_fake

,title,text,subject,date
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017"
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017"
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017"
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017"
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017"
...,...,...,...,...
23476,McPain: John McCain Furious That Iran Treated ...,21st Century Wire says As 21WIRE reported earl...,Middle-east,"January 16, 2016"
23477,JUSTICE? Yahoo Settles E-mail Privacy Class-ac...,21st Century Wire says It s a familiar theme. ...,Middle-east,"January 16, 2016"
23478,Sunnistan: US and Allied ‘Safe Zone’ Plan to T...,Patrick Henningsen 21st Century WireRemember ...,Middle-east,"January 15, 2016"
23479,How to Blow $700 Million: Al Jazeera America F...,21st Century Wire says Al Jazeera America will...,Middle-east,"January 14, 2016"


# Importiamo le funzione necessarie

In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import PassiveAggressiveClassifier
import nltk
nltk.download('stopwords')
import pickle
import string
import spacy
from nltk.corpus import stopwords
import re

english_stopwords = stopwords.words('english')
nlp = spacy.load('en_core_web_sm')
punctuation = set(string.punctuation)
#funzione di cleaning e lemmatizzazione del testo
def data_cleaner(dataset):
    dataset_to_return = []
    if not isinstance(dataset, str):
      for sentence in dataset:
          sentence = sentence.lower()
          for c in string.punctuation:
              sentence = sentence.replace(c, " ")
          document = nlp(sentence)
          sentence = ' '.join(token.lemma_ for token in document)
          sentence = ' '.join(word for word in sentence.split() if word not in english_stopwords)
          sentence = re.sub('\d', '', sentence)
          dataset_to_return.append(sentence)
    else:
      cleaned_sentence = clean_sentence(dataset)
      dataset_to_return.append(cleaned_sentence)

    return dataset_to_return

def clean_sentence(data):
    sentence = data.lower()
    for c in string.punctuation:
        sentence = sentence.replace(c, " ")
    document = nlp(sentence)
    sentence = ' '.join(token.lemma_ for token in document)
    sentence = ' '.join(word for word in sentence.split() if word not in english_stopwords)
    sentence = re.sub('\d', '', sentence)

    return sentence



[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
    else:
      cleaned_sentence = clean_sentence

    return dataset_to_return

def clean_sentence(dataset):

    sentence = dataset.lower()
    for c in string.punctuation:
        sentence = sentence.replace(c, " ")
    document = nlp(sentence)
    sentence = ' '.join(token.lemma_ for token in document)
    sentence = ' '.join(word for word in sentence.split() if word not in english_stopwords)
    sentence = re.sub('\d', '', sentence)

    return dataset


# Fake news per categoria

In [ ]:
fake_news_per_category = df_fake['subject'].value_counts()
fake_news_per_category

News               9050
politics           6841
left-news          4459
Government News    1570
US_News             783
Middle-east         778
Name: subject, dtype: int64

Le categorie "News" e "politics" presentano il maggior numero di fake news.

# I maggiori topic che sono soggetti a fakenews

Iniziamo a definire le varie funzioni per la creazione del modello LDA e per estrarre i topic per ogni categoria. LDA è un modello che sfrutta un algoritmo per trovare topic nascosti in una serie di documenti.

In [ ]:

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation
from collections import Counter

# Creazione del modello di topic modeling
def create_topic_model(dataset, num_topics):
    vectorizer = CountVectorizer()
    X = vectorizer.fit_transform(dataset)

    lda_model = LatentDirichletAllocation(n_components=num_topics, random_state=42)
    lda_model.fit(X)

    return lda_model

# Analisi dei risultati
def extract_top_topics_per_category_2(dataset, subjects, lda_model, num_topics=5):
    top_topics_per_category = {}

    for subject in subjects:
        category_data = dataset[dataset['subject'] == subject]
        category_text = cleaned_data

        category_vectorized = vectorizer.transform(category_text)

        topic_weights = lda_model.transform(category_vectorized)

        avg_topic_weights = topic_weights.mean(axis=0)

        top_topic_indices = avg_topic_weights.argsort()[:-num_topics-1:-1]
        top_topics = [lda_model.components_[i].argsort()[-10:][::-1] for i in top_topic_indices]

        top_topics_per_category[subject] = top_topics

    return top_topics_per_category


Effettuamo prima un cleaning del nostro dataset, togliamo la punteggiatura, le eventuali stopwords ed efftuamo lemmetizzazione, che riporta alla forma base le varie parole.

In [ ]:
subjects = df_fake['subject'].unique()
cleaned_data = data_cleaner(df_fake['text'])

Trasformiamo i testi in vettori e tokeniziamo, addestriamo poi il modello lda per trovare i vari topic

In [ ]:
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(cleaned_data)
num_topics=10
lda_model = LatentDirichletAllocation(n_components=num_topics, random_state=42)
lda_model.fit(X)


LatentDirichletAllocation(random_state=42)

Per ogni categoria troviamo i 5 topic maggiormente soggette a fakenews

In [ ]:
top_topics_per_category = extract_top_topics_per_category(df_fake, subjects, lda_model, num_topics=5)
for subject, top_topics in top_topics_per_category.items():
    print(f"Top topics soggetti a fake news per la categoria '{subject}':")
    for i, topic in enumerate(top_topics):
        words = [vectorizer.get_feature_names_out()[index] for index in topic]
        print(f"Topic {i+1}: {words}")
    print()

Top topics soggetti a fake news per la categoria 'News':
Topic 1: ['trump', 'say', 'donald', 'go', 'get', 'people', 'image', 'like', 'make', 'one']
Topic 2: ['trump', 'president', 'say', 'obama', 'republican', 'right', 'would', 'people', 'court', 'white']
Topic 3: ['state', 'year', 'government', 'say', 'would', 'country', 'million', 'people', 'obama', 'make']
Topic 4: ['say', 'people', 'black', 'right', 'student', 'white', 'one', 'school', 'make', 'like']
Topic 5: ['trump', 'russia', 'news', 'say', 'president', 'medium', 'russian', 'report', 'intelligence', 'election']

Top topics soggetti a fake news per la categoria 'politics':
Topic 1: ['trump', 'say', 'donald', 'go', 'get', 'people', 'image', 'like', 'make', 'one']
Topic 2: ['clinton', 'hillary', 'say', 'state', 'email', 'fbi', 'department', 'investigation', 'campaign', 'report']
Topic 3: ['state', 'year', 'government', 'say', 'would', 'country', 'million', 'people', 'obama', 'make']
Topic 4: ['police', 'say', 'officer', 'gun', 'ma

Attraverso il modello LDA e il calcolo del peso medio di ogni documento, per ogni categoria abbiamo trovato le parole principali dei vari documenti che hanno un peso di importanza maggiore. Dalle parole per ogni categoria, vediamo che alcune si ripetono, ma possiamo comunque intuire i topic, la maggior parte riguardano Trump, o notizie di origine militare.

Proviamo ora a utilizzare sempre LDA model, sul dataset, ma al posto di tokenizzare i singoli documenti, andiamo ad unire il titolo con il testo, e ad analizzare l'intero dataset cercando i topic principali.
iniziamo empre con la pulizia e poi addestriamo il modello LDA

In [ ]:

import gensim
from gensim.utils import simple_preprocess
import nltk
import gensim
from pprint import pprint
import gensim.corpora as corpora
nltk.download('stopwords')
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use'])


In [ ]:
titles = df_fake['title'] +' '+ df_fake['text']

In [ ]:
def sent_to_words(items):
    for item in items:
        # deacc=True removes punctuations
        yield(gensim.utils.simple_preprocess(str(item), deacc=True))

def remove_stopwords(texts):
    return [[word for word in words if word not in stop_words and len(word) >=5] for words in texts]

data_words = list(sent_to_words(titles))
data_words = remove_stopwords(data_words)

In [ ]:
id2word = corpora.Dictionary(data_words)
corpus = [id2word.doc2bow(text) for text in data_words]

In [ ]:
num_topics = 5
lda_model = gensim.models.LdaMulticore(corpus=corpus,
                                       id2word=id2word,
                                       num_topics=num_topics,
                                       passes = 5)
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.014*"clinton" + 0.007*"hillary" + 0.007*"state" + 0.006*"russia" + '
  '0.006*"president" + 0.006*"russian" + 0.005*"court" + 0.005*"obama" + '
  '0.005*"department" + 0.005*"trump"'),
 (1,
  '0.008*"police" + 0.005*"people" + 0.004*"black" + 0.004*"muslim" + '
  '0.004*"would" + 0.004*"video" + 0.003*"group" + 0.003*"school" + '
  '0.003*"media" + 0.003*"students"'),
 (2,
  '0.012*"obama" + 0.010*"trump" + 0.010*"president" + 0.008*"would" + '
  '0.006*"people" + 0.006*"states" + 0.005*"state" + 0.005*"american" + '
  '0.005*"america" + 0.004*"united"'),
 (3,
  '0.030*"trump" + 0.016*"hillary" + 0.012*"clinton" + 0.010*"media" + '
  '0.009*"video" + 0.007*"donald" + 0.006*"president" + 0.006*"campaign" + '
  '0.005*"would" + 0.005*"watch"'),
 (4,
  '0.050*"trump" + 0.015*"twitter" + 0.014*"president" + 0.010*"donald" + '
  '0.010*"people" + 0.010*"white" + 0.009*"video" + 0.008*"black" + '
  '0.006*"obama" + 0.005*"america"')]


Come possiamo vedere, i risultati sono molto simili, i topic principali sono politici e militari

# Pattern nei titoli

Per trovare dei pattern possiamo provare ad analizzare le sequenze di N-grammi più frequenti.
Come prima iniziamo con la pulizzia,  lemmatizzazzione e suddivisione in n-grammi

In [ ]:
title_clean = data_cleaner(df_fake['title'])


In [ ]:
df_fake['title_clean'] = data_cleaner(df_fake['title'])

In [ ]:
vectorizer = CountVectorizer(ngram_range=(2, 2))
X = vectorizer.fit_transform(df_fake['title_clean'])
feature_names = vectorizer.get_feature_names_out()
feature_names

array(['aaron roger', 'aarp catch', 'abandon foreign', ..., 'état claim',
       'état constitution', 'žižek american'], dtype=object)

In [ ]:
ngram_frequencies = X.sum(axis=0).A1
ngram_freq_df = pd.DataFrame({'n-grammi': feature_names, 'frequenza': ngram_frequencies})
ngram_freq_df = ngram_freq_df.sort_values(by='frequenza', ascending=False)
ngram_freq_df.head(10)

,n-grammi,frequenza
28247,donald trump,805
73785,president trump,497
44101,hillary clinton,412
101092,trump supporter,405
108544,white house,348
36535,fox news,314
101276,trump video,289
4200,anti trump,261
47024,illegal alien,215
32908,fake news,207


In [ ]:
vectorizer = CountVectorizer(ngram_range=(3, 3))
X = vectorizer.fit_transform(df_fake['title_clean'])
feature_names = vectorizer.get_feature_names_out()
feature_names

array(['aaron roger ask', 'aarp catch use', 'abandon foreign policy', ...,
       'zuckerberg turn congress', 'état claim cold',
       'žižek american leave'], dtype=object)

In [ ]:
ngram_frequencies = X.sum(axis=0).A1
ngram_freq_df = pd.DataFrame({'n-grammi': feature_names, 'frequenza': ngram_frequencies})
ngram_freq_df = ngram_freq_df.sort_values(by='frequenza', ascending=False)
ngram_freq_df.head(10)

,n-grammi,frequenza
11538,black life matter,161
12651,boiler room ep,138
114836,supreme court justice,53
78815,new york times,51
43548,fox news host,49
33683,donald trump jr,47
33863,donald trump video,45
4941,anti trump protester,42
122120,trump campaign manager,41
139499,year old girl,32


# Modello di riconoscimento Fake news

Iniziamo con l'aggiungere una nuopva colonna nei due data set c con valori 0 e 1, il primo saranno le news veritiere e le seconde le fake news.

In [ ]:
df_fake['label'] = 1
df_true['label'] = 0

Uniamo i due data set in un unico dataset. Dividiamo poi in dataset di training e testing, effetuamo poi la pulizia

In [ ]:
dataset = pd.concat([df_fake, df_true], ignore_index=True)
dataset['cleaned_text'] = data_cleaner(dataset['text'])
X_train, X_test, y_train, y_test = train_test_split(dataset['cleaned_text'], dataset['label'], test_size=0.2, random_state=42)

Addestriamo poi il modello Naive-Bayes per il riconoscimento di fake news

In [ ]:
from sklearn.naive_bayes import MultinomialNB
vectorizer = TfidfVectorizer()
X_train_text_features = vectorizer.fit_transform(X_train)
X_test_text_features = vectorizer.transform(X_test)

# Creazione del modello
model = MultinomialNB()
model.fit(X_train_text_features, y_train)

MultinomialNB()

In [ ]:
model.score(X_test_text_features, y_test)

0.9373051224944321

Dopo aver addestrato il modello con i dati di training, il test di accuratezza è del 0.93 un valore alto, direi che abbiamo un buon modello per controllare le fake news.

Definiamo ora la funzione per il controllo delle varie news.

In [ ]:
def predict_news(title, text):
    cleaned_text = data_cleaner(text)
    cleaned_text_string = ' '.join(cleaned_text)
    new_text = vectorizer.transform([cleaned_text_string])
    prediction = model.predict(new_text)
    if prediction[0] == 1:
      print(f'La notizia:"{title}"  è falsa')
    else:
      print(f'La notizia:"{title}"  è vera')



#Esempio

Importiamo un nuovo dataset con sia notizie vere e false.
Questo data set proviene da [kaggle](https://www.kaggle.com/competitions/fake-news/data?select=train.csv)

In [ ]:
df_2 = pd.read_csv("train.csv")
df_2

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1
...,...,...,...,...,...
20795,20795,Rapper T.I.: Trump a ’Poster Child For White S...,Jerome Hudson,Rapper T. I. unloaded on black celebrities who...,0
20796,20796,"N.F.L. Playoffs: Schedule, Matchups and Odds -...",Benjamin Hoffman,When the Green Bay Packers lost to the Washing...,0
20797,20797,Macy’s Is Said to Receive Takeover Approach by...,Michael J. de la Merced and Rachel Abrams,The Macy’s of today grew from the union of sev...,0
20798,20798,"NATO, Russia To Hold Parallel Exercises In Bal...",Alex Ansary,"NATO, Russia To Hold Parallel Exercises In Bal...",1


Andiamo a selezionare una news casuale dal nostro dataset.

In [ ]:
new_title = df_2.loc[8293]['title']
new_text = df_2.loc[8293]['text']

predict = predict_news(new_title, new_text)

La notizia:"Michael Moore Wants A Spot in Hillary’s Cabinet"  è falsa


In [ ]:
new_text = df_2.loc[8293]['label']
new_text

1

Il nostro modello ha riconosciuto la notizia falsa

In [ ]:
new_title = df_2.loc[2239]['title']
new_text = df_2.loc[2239]['text']

predict = predict_news(new_title, new_text)

La notizia:"U.S. Presidential Race, North Dakota, NASA: Your Friday Evening Briefing - The New York Times"  è falsa


# Esportazione del modello

In [ ]:
import pickle
with open('Hu_fake_filters','wb') as file:
  pickle.dump(model, file)